In [22]:
from sklearn import linear_model
import math
import numpy as np
import matplotlib.pyplot as plt
import gzip
import pickle
import pandas as pd
from scipy.stats import norm
from scipy.optimize import least_squares


def bazin(time, A, B, t0, tfall, trise):
    X = np.exp(-(time - t0) / tfall) / (1 + np.exp((time - t0) / trise))
    return A * X + B

def lightcurve_fit(time, flux):
    scaled_time = time - time.min()
    t0 = scaled_time[flux.argmax()]
    guess = (0, 0, t0, 40, -5)
    errfunc = lambda params: abs(flux - bazin(scaled_time, *params))
    result = least_squares(errfunc, guess, method='lm')
    return result.x

def predict (x, deg, coef):
    p = 1
    result = 0
    for i in range(deg + 1):
        result += p*coef[deg-i]
        p = p*x
    return result
def predicts (x_vect, deg, coef):
    result = []
    for x in x_vect:
        result.append(predict(x, deg, coef))
    return result

def error(y1, y2):
    a = 0
    for i in range(len(y1)):
        a += (y1[i]-y2[i])**2
    return math.sqrt(a)

filename = "data1/des_train_mini.pkl"
data = None
with gzip.open(filename, 'rb') as f:
    data = pickle.load(f)

deg = 3
errors = pd.Series({'g_flux':0,'r_flux':0,'i_flux':0,'z_flux':0, 'g_fluxe':0,'r_fluxe':0,'i_fluxe':0,'z_fluxe':0})
stats = pd.DataFrame(({'g':[],'r':[],'i':[],'z':[]}))
l = len(data)
erreurs = pd.DataFrame()
x1, x21, y11, y21 = None, None, None, None
Y, Ye = None, None
mie, miv = 1e30000, 1e3000


x13, x213, y113, y213 = None, None, None, None
Y3, Ye3 = None, None
mie3, miv3 = 0, 0

a = 0
for idx in data:
    t = pd.Series()
    sd =  pd.Series()#{'g_flux':0,'r_flux':0,'i_flux':0,'z_flux':0, 'g_fluxe':0,'r_fluxe':0,'i_fluxe':0,'z_fluxe':0})
    try:
        for band in 'griz':
            d = data[idx][band]
            x_train = np.array(d['mjd'])
            t[band] = len(x_train)
            x = x_train
            #x_train = x_train.reshape(-1, 1)
            y_traine = np.array(d['fluxcalerr'])
            y_train = np.array(d['fluxcal'])

            #fit = lightcurve_fit(x_train, y_train)
            #fite = lightcurve_fit(x_train, y_traine)
            #v = bazin(stim2, *fit)
            #e = bazin(stim2, *fite)
            ab = np.polyfit(x_train, y_train, deg = deg)
            bb = np.polyfit(x_train, y_traine, deg = deg)
            
            """
            #model linear
            regr = linear_model.LinearRegression()
            regre = linear_model.LinearRegression()
            regr.fit(x_train, y_train)
            regre.fit(x_train, y_traine)
            """
            #y1, y2 = regr.predict(x_train), regre.predict(x_train)
            #y1, y2 = bazin(x_train - x_train.min(),*fit), bazin(x_train - x_train.min(),*fite)
            y1, y2 = predicts(x_train, deg, ab), predicts(x_train, deg, bb)
            ve = error(y1, y_train)
            ee = error(y2, y_traine)
            if ve < miv:
                x1 = x
                y11 = y1
                Y = y_train
                miv = ve
            if ee < mie:
                x21 = x
                y21 = y2
                Ye = y_traine
                mie = ee

            if ve > miv3:
                x13 = x
                y113 = y1
                Y3 = y_train
                miv3 = ve
            if ee > mie3:
                x213 = x
                y213 = y2
                Ye3 = y_traine
                mie3 = ee   
            errors[band+"_flux"] = errors[band+"_flux"] + ve
            errors[band+"_fluxe"] = errors[band+"_fluxe"] + ee
            sd[band+"_flux"] = ve
            sd[band+"_fluxe"] = ee
    except:
        a = a + 1
    stats = stats.append(t, ignore_index = True)
    erreurs = erreurs.append(sd, ignore_index = True)
    #break
print(a)

print(l)
print(errors) 
#print(stats)
print(erreurs)
errors.to_csv("errors_poly3.csv")
#stats.to_csv("stats.csv")
erreurs.to_csv("erreurs_poly3.csv")


plt.plot(x1, Y, 'o')
plt.plot(x1, y11)
plt.title(band)
plt.savefig("imgvalue_poly3.png")
plt.show()
plt.plot(x21, Ye, 'o')
plt.plot(x21, y21)
plt.title(band+"e")
plt.savefig("imgerreur_poly3.png")
plt.show()


plt.plot(x13, Y3, 'o')
plt.plot(x13, y113)
plt.title(band)
plt.savefig("imgvaluemax_poly3.png")
plt.show()
plt.plot(x213, Ye3, 'o')
plt.plot(x213, y213)
plt.title(band+"e")
plt.savefig("imgerreurmax_poly3.png")
plt.show()#"



0
300
g_flux     21538
r_flux     22322
i_flux     22503
z_flux     18615
g_fluxe     6848
r_fluxe     3010
i_fluxe     4121
z_fluxe     3379
dtype: int64
         g_flux    g_fluxe      i_flux    i_fluxe      r_flux    r_fluxe  \
0     39.432071  34.016879   21.561253  12.821849   17.588945  10.274587   
1     11.003387   5.668156   20.996880   3.076891   22.245347   1.937104   
2    109.187628  35.101592  129.236273  12.704664  149.590423  10.296427   
3     56.208222  27.756921   67.951332  24.654397   36.221442  13.562841   
4    150.913177  21.961650   82.606680   8.597849   94.839835   5.022493   
5     81.590253  21.858600   69.653518   9.875740   53.810635   5.552028   
6     39.591715  21.701772   32.792606   9.992539   30.109327   5.723628   
7     41.555263  26.480123   32.659705  24.072549   22.556649  13.064212   
8     30.376308  10.021865   58.432971   7.998127   59.890603   3.465707   
9     55.064445   8.723854   53.187189   2.263532   48.647779   2.260320   
10     1.

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

In [24]:
means = pd.Series()
stds = pd.Series()
means1 = pd.Series()
stds1 = pd.Series()

for ind in stats.columns:
    means[ind] = np.mean(stats[ind])
    stds[ind] = np.std(stats[ind])
for ind in erreurs.columns:
    means1[ind] = np.mean(erreurs[ind])
    stds1[ind] = np.std(erreurs[ind])
print("statistques:")
print("mean: ")
print(means)
print("std: ")
print(stds)
print("erreurs: \n")
print("mean: \n")
print(means1)
print("std: \n")
print(stds1)

statistques:
mean: 
g    21.290000
r    20.766667
i    20.250000
z    19.703333
dtype: float64
std: 
g    8.854334
r    8.687091
i    8.332317
z    8.195242
dtype: float64
erreurs: 

mean: 

g_flux     72.284646
g_fluxe    23.300466
i_flux     75.529439
i_fluxe    14.243250
r_flux     74.896889
r_fluxe    10.531774
z_flux     62.541965
z_fluxe    11.774975
dtype: float64
std: 

g_flux     105.169204
g_fluxe     15.355964
i_flux      83.016597
i_fluxe     12.217352
r_flux     102.101572
r_fluxe     10.894790
z_flux      65.065822
z_fluxe     10.350860
dtype: float64


In [21]:
"""
import numpy as np
def predict (x, deg, coef):
    p = 1
    result = 0
    for i in range(deg + 1):
        result += p*coef[deg-i]
        p = p*x
    return result

deg = 3
a = np.polyfit([1,2,3,4,5,6,7, 8, 9, 10], [1, 4, 9, 16, 25, 36, 49, 64, 81, 100], deg)
print(predict(0.1, deg, a))"""

0.009999999999991512


In [4]:
1.00000000e+00*2*2+2.31547410e-15*2-2.11304763e-14

3.999999999999983